In [2]:
import json
import csv
import os
import pandas as pd

In [11]:
# Create the header row and the function that appends rows to the row_list

row_list = [['state', 'tract', 'coordinates']]
def create_csv_file(name, in_path):  
    with open(in_path, 'r+') as f:
        data = json.load(f)
        tracts = data['features']
        for tract in tracts:
            output_arr = []
            num = tract['properties']['STATEFP'] + tract['properties']['COUNTYFP'] + tract['properties']['TRACTCE']
            if num in ['02240000100', '02290000100', '02290000200', '02290000300']:
                continue
            coords = tract['geometry']['coordinates'][0]
            if tract['geometry']['type'] == 'MultiPolygon':
                for outer in coords:
                    for inner in outer:
                        inner.reverse()
            else:
                for a in coords:
                    a.reverse()
            output_arr.append(name)
            output_arr.append(num)
            output_arr.append(str(coords))
            row_list.append(output_arr)

In [12]:
# Loop over all geojson files and pass to the function above
geo_path = r'C:\Users\kyle2\Downloads\2020_tract_files\2020_tract_files-main\GeoJsonFiles'

lst_of_geoJson = os.listdir(geo_path)

for i in lst_of_geoJson:
    i_path = geo_path + '\\' + i
    name = i[:len(i) - 5]
    create_csv_file(name, i_path)

In [5]:
# output the row_list which now contains all data from the geojson files to a csv file

with open('tract_poly_with_multipolygons.csv', 'w') as file:
     writer = csv.writer(file)
     writer.writerows(row_list)

### Random Analysis

In [6]:
df = pd.read_csv('tract_poly_with_multipolygons.csv', dtype={'tract': 'string', 'coordinates':'string'})

In [9]:
df.head()

,state,tract,coordinates
0,Alabama,01001020100,"[[32.505159, -86.488262], [32.503739, -86.4893..."
1,Alabama,01001020200,"[[32.489342, -86.478867], [32.48597, -86.47945..."
2,Alabama,01001020300,"[[32.489203, -86.46731], [32.485557, -86.46945..."
3,Alabama,01001020400,"[[32.480213, -86.435995], [32.482863, -86.4363..."
4,Alabama,01001020501,"[[32.460461, -86.434115], [32.459476, -86.4350..."


In [8]:
df[df['coordinates'].str.len() == df['coordinates'].str.len().max()]

,state,tract,coordinates
77420,Utah,49037978200,"[[37.541063, -110.646394], [37.538394, -110.64..."


In [10]:
df[df['tract'] == '01071950700']

,state,tract,coordinates
464,Alabama,01071950700,"[[[34.533633, -86.149806], [34.533634, -86.149..."


In [94]:
df[df['coordinates'].str.len() >= 65535]['tract'].tolist()

[]

In [82]:
len(df)

84122

In [81]:
df.dtypes

state          object
tract          string
coordinates    string
dtype: object

In [86]:
df.loc[1609]

state                                                     Alaska
tract                                                02290000100
coordinates    [[67.999999, -149.32543], [67.998816, -149.326...
Name: 1609, dtype: object